In [1050]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import datasets, model_selection, preprocessing, decomposition

In [1051]:
X_y_dict = dict()

In [1052]:
cancer_dataset = datasets.load_breast_cancer()
# cancer_data = pd.DataFrame(data=cancer_dataset.data, columns=cancer_dataset.feature_names)

# label all items in the dataset
# cancer_data['type'] = [cancer_dataset.target_names[idx] for idx in cancer_dataset.target]
# cancer_data.head() # the last column "type" was added

cancer_X = cancer_dataset.data
cancer_y = cancer_dataset.target

# cancer_X_recip = cancer_X.copy()
# m, n = cancer_X_recip.shape
# for i in range(m):
#     for j in range(n):
#         value = cancer_X_recip[i][j]
#         cancer_X_recip[i][j] = (1.0 / value) if value != 0 else 0.

# cancer_X = np.hstack((cancer_X, cancer_X_recip))

X_y_dict['cancer'] = [cancer_X, cancer_y]

In [1053]:
iris_dataset = datasets.load_iris()
# iris_data = pd.DataFrame(data=iris_dataset.data, columns=iris_dataset.feature_names)

# label all items in the dataset
# iris_data['type'] = [iris_dataset.target_names[idx] for idx in iris_dataset.target]
# iris_data.head() # the last column "type" was added

iris_X = iris_dataset.data 
iris_y = iris_dataset.target

# iris_X_recip = iris_X.copy()
# m, n = iris_X_recip.shape
# for i in range(m):
#     for j in range(n):
#         value = iris_X_recip[i][j]
#         iris_X_recip[i][j] = (1.0 / value) if value != 0 else 0.

# iris_X = np.hstack((iris_X, np.power(iris_X, 2), iris_X_recip, np.power(iris_X_recip, 2)))

        
X_y_dict['iris'] = [iris_X, iris_y]

In [1054]:
testdata = np.loadtxt('testdata.csv')
testdata_X = testdata[:, 0:-1]
testdata_y = testdata[:, -1]

X_y_dict['testdata'] = [testdata_X, testdata_y]

In [1055]:
# heart_df = pd.read_csv('heart.csv')

# first = pd.get_dummies(heart_df['cp'], prefix="cp")
# second = pd.get_dummies(heart_df['slope'], prefix="slope")
# thrid = pd.get_dummies(heart_df['thal'], prefix="thal")

# heart_df = heart_df.drop(columns = ['cp', 'slope', 'thal'])
# heart_df = pd.concat([first, second, thrid, heart_df], axis=1)

# heart_df.head(3)


In [1056]:
heart_df = pd.read_csv('heart.csv')

features_to_norm = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'ca']

cp = pd.get_dummies(heart_df['cp'], prefix="cp")
slope = pd.get_dummies(heart_df['slope'], prefix="slope")
thal = pd.get_dummies(heart_df['thal'], prefix="thal")
heart_df = heart_df.drop(columns = ['cp', 'slope', 'thal'])

heart_df_p1 = heart_df[features_to_norm]
heart_df_p2 = heart_df.drop(columns = features_to_norm)
# print(heart_df_p1.loc[0])

heart_X_p1 = heart_df_p1.to_numpy()

heart_X_p1_recip = heart_X_p1.copy()
m, n = heart_X_p1_recip.shape
for i in range(m):
    for j in range(n):
        value = heart_X_p1_recip[i][j]
        heart_X_p1_recip[i][j] = (1.0 / value) if value != 0 else 0.
#         heart_X_p1_recip[i][j] = sigmoid(value)
# print(heart_X_p1_recip[0])

# heart_X_p1 = np.hstack((heart_X_p1, np.power(heart_X_p1, 2)))
# heart_X_p1 = np.hstack((heart_X_p1, np.power(heart_X_p1, 2), heart_X_p1_recip))
heart_X_p1 = np.hstack((heart_X_p1, heart_X_p1_recip))
# heart_X_p1 = np.hstack((heart_X_p1, np.power(heart_X_p1, 2), heart_X_p1_recip, np.power(heart_X_p1_recip, 2)))
# heart_X_p1 = np.hstack((heart_X_p1, np.power(heart_X_p1, 2), np.power(heart_X_p1, 3), heart_X_p1_recip))
# print(heart_X_p1_recip[0])

ss = preprocessing.StandardScaler()
heart_X_p1 = ss.fit_transform(heart_X_p1)
# print(heart_X_p1[0])

heart_df_p2 = pd.concat([cp, slope, thal, heart_df_p2], axis=1)
heart_X_p2 = heart_df_p2.to_numpy()[:, 0:-1]
heart_y = heart_df_p2.to_numpy()[:, -1]
# print(heart_X_p2[0])


heart_X = np.hstack((heart_X_p1, heart_X_p2))

PCA = decomposition.PCA(10)
new_heart_X = PCA.fit_transform(heart_X)

X_y_dict['heart'] = [new_heart_X, heart_y]
# print(heart_df.loc[0])
# print(heart_X[0])
# print(new_heart_X[0])

In [1066]:
[X, y] = X_y_dict['cancer']


feature_amount = X.shape[1]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.25)

# features_to_norm = ['age', 'trestbps', 'chol', 'tahlach', 'oldpeak', 'ca']
# ss = preprocessing.StandardScaler()
# X_train[features_to_norm] = ss.fit_transform(X_train[features_to_norm])
# X_test[features_to_norm] = ss.transform(X_test[features_to_norm])

train_ones = np.ones((X_train.shape[0], 1))
X_train = np.hstack((X_train, train_ones))

test_ones = np.ones((X_test.shape[0], 1))
X_test = np.hstack((X_test, test_ones))

y_train = y_train.reshape((y_train.shape[0], 1))
y_test = y_test.reshape((y_test.shape[0], 1))

# print(X_train.shape)
# print(y_train.shape)
# print(X_test.shape)
# print(y_test.shape)
# print(feature_amount)
# print(X_train[0])

In [1067]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l2', solver='newton-cg', multi_class='auto', max_iter=2000)
lr.fit(X_train,y_train.ravel())

print("Logistic Regression模型训练集的准确率：%.3f" %lr.score(X_train, y_train.ravel()))
print("Logistic Regression模型测试集的准确率：%.3f" %lr.score(X_test, y_test.ravel()))
from sklearn import metrics
y_hat = lr.predict(X_test)
accuracy = metrics.accuracy_score(y_test.ravel(), y_hat) #错误率，也就是np.average(y_test==y_pred)
print("Logistic Regression模型正确率：%.3f" %accuracy)

Logistic Regression模型训练集的准确率：0.972
Logistic Regression模型测试集的准确率：0.944
Logistic Regression模型正确率：0.944


In [788]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

def predict_prob(theta, X):
    z = X.dot(theta)
    return sigmoid(z)

def predict(y_pred_prob):
    y_pred = [([1.] if item >= 0.5 else [0.]) for item in y_pred_prob]
    return np.array(y_pred)

def get_cost_value(y, y_pred_prob):
    sample_amount = y.shape[0]
    value_sum = 0
    for i in range(sample_amount):
        value_sum += y[i] * np.log(y_pred_prob[i]) \
                     + (1 - y[i]) * np.log(1 - y_pred_prob[i])
    return (-1.0 / sample_amount) * value_sum[0] 

def get_correct_pred_num(y, y_pred):
    count = 0
    for i in range(y_pred.shape[0]):
        if y_pred[i][0] == y[i][0]:
            count += 1 
    return count

def get_accuracy(y, y_pred_prob):
    count = get_correct_pred_num(y, y_pred_prob)
    return count / y.shape[0]

def update_BGD(theta, X, y, y_pred_prob, alpha=0.005):
    sample_amount = X.shape[0]
    dtheta = X.T.dot(y - y_pred_prob) / sample_amount
    theta = theta + alpha * dtheta
    return theta 

def update_SGD(theta, X, y, y_pred_prob, alpha=0.005):
    for i in range(X.shape[0]):
        dtheta = X[i] * (y[i] - y_pred_prob[i])
        for j in range(feature_amount+1):
            theta[j][0] += alpha * dtheta[j]
    return theta
    
def train(theta, X, y, iterations=1000):
    cost_value_record = []
    accuracy_record = []
    theta_record = []
    update_func = update_SGD
    
    for i in range(iterations):
        y_pred_prob = predict_prob(theta, X)
        cost_value = get_cost_value(y, y_pred_prob)
        accuracy = get_accuracy(y, predict(y_pred_prob))
        # print(i, accuracy, cost_value)

        theta = update_func(theta, X, y, y_pred_prob)
                
        cost_value_record.append(cost_value)
        accuracy_record.append(accuracy)
        theta_record.append(theta.copy())
        
        if i % 200 == 0:
            y_test_pred_prob = predict_prob(theta, X_test)
            y_test_pred = predict(y_test_pred_prob)
            print('iteration', i, 'finished. Acc: ', accuracy, get_accuracy(y_test, y_test_pred))

    
    return theta, cost_value_record, accuracy_record, theta_record

In [703]:
init_theta = np.random.randn(feature_amount+1, 1)
# print(init_theta)

final_theta, cost_values, accs, thetas = train(init_theta.copy(), X_train, y_train, 5000)
# print(cost_values[-1], accs[-1])

y_test_pred_prob = predict_prob(final_theta, X_test)
y_test_pred = predict(y_test_pred_prob)
# print(y_test_pred)
# print(y_test)

count = get_correct_pred_num(y_test, y_test_pred)
print(count, y_pred.shape[0], get_accuracy(y_test, y_test_pred))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in multiply


iteration 0 finished. Acc:  0.6123348017621145 0.6578947368421053
iteration 200 finished. Acc:  0.8414096916299559 0.8026315789473685
iteration 400 finished. Acc:  0.8546255506607929 0.8026315789473685
iteration 600 finished. Acc:  0.8281938325991189 0.7368421052631579
iteration 800 finished. Acc:  0.8370044052863436 0.8157894736842105
iteration 1000 finished. Acc:  0.8237885462555066 0.7368421052631579
iteration 1200 finished. Acc:  0.8237885462555066 0.7368421052631579
iteration 1400 finished. Acc:  0.8281938325991189 0.7368421052631579
iteration 1600 finished. Acc:  0.8458149779735683 0.7763157894736842
iteration 1800 finished. Acc:  0.762114537444934 0.7894736842105263
iteration 2000 finished. Acc:  0.8546255506607929 0.7894736842105263
iteration 2200 finished. Acc:  0.8370044052863436 0.8421052631578947
iteration 2400 finished. Acc:  0.7665198237885462 0.7894736842105263
iteration 2600 finished. Acc:  0.7577092511013216 0.7894736842105263
iteration 2800 finished. Acc:  0.841409691

In [710]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l2', solver='newton-cg', multi_class='auto', max_iter=2000)
lr.fit(X_train,y_train.ravel())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=2000, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [711]:
print("Logistic Regression模型训练集的准确率：%.3f" %lr.score(X_train, y_train.ravel()))
print("Logistic Regression模型测试集的准确率：%.3f" %lr.score(X_test, y_test.ravel()))
from sklearn import metrics
y_hat = lr.predict(X_test)
accuracy = metrics.accuracy_score(y_test.ravel(), y_hat) #错误率，也就是np.average(y_test==y_pred)
print("Logistic Regression模型正确率：%.3f" %accuracy)

Logistic Regression模型训练集的准确率：0.850
Logistic Regression模型测试集的准确率：0.868
Logistic Regression模型正确率：0.868
